In [1]:
%%time

import json

import pandas
import pandas as pd
import requests
from PIL import Image
from adlfs import AzureBlobFileSystem
from tqdm import tqdm

from common.captioning.azure_descriptions import AzureCaption
from common.storage.azure_file_storage import AzureFileStorageAdapter

tqdm.pandas(desc="Progress")
file_system: AzureBlobFileSystem = AzureFileStorageAdapter('data').get_file_storage()

from common.functions.functions import Functions

functions: Functions = Functions()

caption: AzureCaption = AzureCaption(file_system)

CPU times: total: 9.67 s
Wall time: 16.7 s


In [2]:
%%time

curated_df = pandas.read_parquet('data/parquet/back.parquet', filesystem=file_system, engine='pyarrow')

accepted = curated_df.loc[curated_df["accept"] == True]

accepted.dropna(inplace=True)

accepted.reset_index(inplace=True, drop=True)

display(accepted)

<timed exec>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
0,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
1,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
2,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
3,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]
4,105styc,AmIhotAF,Gizzygirl127,Low key… still bangable?,smiling woman sitting on couch with remote con...,6d555943be4fbc21ff92417c6f582298,/r/AmIhotAF/comments/105styc/low_key_still_ban...,https://i.redd.it/aiaxxoz9uoaa1.jpg,105styc.jpg,data/image/105styc.jpg,SexyDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19899,13m6hv4,tightdresses,katiecooper4xo,Bringing tartan back!,arafed woman in a plaid dress posing for a pic...,9501d5f34d0748f58eb3caba7ce1641c,/r/tightdresses/comments/13m6hv4/bringing_tart...,https://i.redd.it/8nx1kv7o0w0b1.jpg,13m6hv4.jpg,data/image/13m6hv4.jpg,PrettyGirlDiffusion,True,True,True,[]
19900,13mcqn4,tightdresses,GucciBagHolder,Blonde,a close up of a woman in a white dress posing ...,70b0eff895fd87d368cb1f53af9a7f3c,/r/tightdresses/comments/13mcqn4/blonde/,https://i.redd.it/6ojq4wtl9x0b1.jpg,13mcqn4.jpg,data/image/13mcqn4.jpg,PrettyGirlDiffusion,True,True,True,[]
19901,13m91m2,DLAH,qwertybird3141,College grad,arafed woman standing on steps with a red and ...,adcbd61b5e37c5e048cb6907eca21bfe,/r/DLAH/comments/13m91m2/college_grad/,https://i.redd.it/skv5m0jwiw0b1.jpg,13m91m2.jpg,data/image/13m91m2.jpg,PrettyGirlDiffusion,True,True,True,[]
19902,13m6gm7,bathandbodyworks,unicornkenz,Candle Sale,bath and body works coup coup,47a78b117b59b636fa22093ec4f29bd7,/r/bathandbodyworks/comments/13m6gm7/candle_sale/,https://i.redd.it/h321gmgf0w0b1.jpg,13m6gm7.jpg,data/image/13m6gm7.jpg,CandleDiffusion,True,True,True,[]


CPU times: total: 1.03 s
Wall time: 3.82 s


In [3]:
%%time

captions = pd.read_parquet('data/parquet/image_captions.parquet', filesystem=file_system, engine='pyarrow')
tags = pd.read_parquet('data/parquet/image_tags.parquet', filesystem=file_system, engine='pyarrow')
crops = pd.read_parquet('data/parquet/image_cropping.parquet', filesystem=file_system, engine='pyarrow')

display(captions.shape)
display(tags.shape)
display(crops.shape)

(59102, 7)

(121919, 3)

(6614, 6)

CPU times: total: 500 ms
Wall time: 1.34 s


In [4]:
%%time

current_captions = file_system.ls("data/caption")
display(len(current_captions))

6732

CPU times: total: 10.2 s
Wall time: 16.3 s


In [5]:
%%time

all_data = []
for caption_file in tqdm(current_captions, total=len(current_captions), desc='Reading caption files'):
	try:
		caption_data = json.loads(file_system.read_text(caption_file, encoding='utf-8'))
		dense_caption_result = caption_data.get('denseCaptionsResult')
		metadata = caption_data.get('metadata')
		tags_result = caption_data.get('tagsResult')
		smart_crop_result = caption_data.get('smartCropsResult')
		basic_caption = caption_data.get('captionResult')
		image_id = caption_file.split('/')[-1].split('.')[0]
		caption_data["id"] = image_id
		filtered_data = {
			"id": image_id,
			"captions": [basic_caption],
			"dense_captions": dense_caption_result['values'],
			"meta": [metadata],
			"tags": tags_result['values'],
			"smart_crop": smart_crop_result['values']
		}
		all_data.append(filtered_data)
	except Exception as e:
		print(e)
		continue

Reading caption files:  67%|██████▋   | 4479/6732 [06:03<02:49, 13.26it/s]

Expecting value: line 1 column 1 (char 0)


Reading caption files:  67%|██████▋   | 4493/6732 [06:04<02:43, 13.69it/s]

Expecting value: line 1 column 1 (char 0)


Reading caption files:  67%|██████▋   | 4501/6732 [06:04<02:29, 14.94it/s]

...

Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)


Reading caption files:  71%|███████   | 4784/6732 [06:22<01:32, 21.03it/s]

Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)


Reading caption files: 100%|██████████| 6732/6732 [09:09<00:00, 12.25it/s]


CPU times: total: 2min 40s
Wall time: 9min 9s


In [6]:
%%time

new_captions = pandas.json_normalize(data=all_data, record_path=['dense_captions'], meta=['id'],
									 record_prefix='dense_captions_')
new_tags = pandas.json_normalize(data=all_data, record_path=['tags'], meta=['id'], record_prefix='tags_')
new_crops = pandas.json_normalize(data=all_data, record_path=['smart_crop'], meta=['id'], record_prefix='smart_crop_')

CPU times: total: 7.36 s
Wall time: 19.7 s


In [7]:
%%time

new_basic_captions = pandas.json_normalize(data=all_data, record_path=['captions'], meta=['id'],
										   record_prefix='captions_')
meta = pandas.json_normalize(data=all_data, record_path=['meta'], meta=['id'], record_prefix='meta_')

CPU times: total: 297 ms
Wall time: 559 ms


In [8]:
%%time

display(new_captions.shape)
display(new_tags.shape)
display(new_crops.shape)
display(new_basic_captions.shape)
display(meta.shape)

(59102, 7)

(121919, 3)

(6614, 6)

(6614, 3)

(6614, 3)

CPU times: total: 15.6 ms
Wall time: 15 ms


In [9]:
%%time

merge_singles = pandas.merge(new_basic_captions, meta, on='id').set_index(keys=['id'], drop=False)
merge_singles.drop_duplicates(inplace=True)
merge_singles.reset_index(drop=True, inplace=True)
display(merge_singles)

,captions_text,captions_confidence,id,meta_width,meta_height
0,a pier in the ocean,0.321403,1002bsl,4032,3024
1,a town in the mountains,0.314388,1002cgj,1200,800
2,a woman taking a selfie,0.662057,1002cx2,1468,2608
3,a woman in a garment standing next to a tree,0.513081,1003dod,1523,2030
4,a train on a bridge over a body of water,0.382730,10044wv,4032,3024
...,...,...,...,...,...
6609,a woman in a blue lingerie,0.430763,zzu8i8,2309,2560
6610,a city with many buildings and a road,0.255758,zzv4hz,1283,1519
6611,a woman sitting on a bed,0.584322,zzvv3x,1440,1800
6612,a woman in a dress posing for a picture,0.460149,zzxq2l,1075,1351


CPU times: total: 31.2 ms
Wall time: 165 ms


In [10]:
%%time

merged_to_curate = pandas.merge(merge_singles, curated_df, on='id', how='outer').set_index(keys=['id'], drop=False)
merged_to_curate.fillna(value='', inplace=True)
display(merged_to_curate)

,captions_text,captions_confidence,id,meta_width,meta_height,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,,,,,
1002bsl,a pier in the ocean,0.321403,1002bsl,4032.0,3024.0,CityPorn,Bottomsup99,Pacific Beach California,arafed view of a pier with a pier and a pier,08036f856ed7759eb73ebd6b077ea642,/r/CityPorn/comments/1002bsl/pacific_beach_cal...,https://i.redd.it/slyqoecbzb9a1.jpg,1002bsl.jpg,data/image/1002bsl.jpg,CityDiffusion,True,True,True,[]
1002cgj,a town in the mountains,0.314388,1002cgj,1200.0,800.0,CityPorn,drizzydriller,"Not exactly a city, but Jim Thorpe PA is very ...",mountainside view of a town with a flagpole an...,a3d715cfdbb49376fbabfb29a1bd6400,/r/CityPorn/comments/1002cgj/not_exactly_a_cit...,https://i.redd.it/09kuna4hzb9a1.jpg,1002cgj.jpg,data/image/1002cgj.jpg,CityDiffusion,True,True,True,[]
1002cx2,a woman taking a selfie,0.662057,1002cx2,1468.0,2608.0,SFWRedheads,Sayleywayley,Happy New Year to every single person near and...,blonde woman with red hair and black bra top p...,1fb785ca16b10f4f7613961f0b88f369,/r/SFWRedheads/comments/1002cx2/happy_new_year...,https://i.redd.it/el3s490lzb9a1.jpg,1002cx2.jpg,data/image/1002cx2.jpg,RedHeadDiffusion,True,True,True,[]
1003dod,a woman in a garment standing next to a tree,0.513081,1003dod,1523.0,2030.0,HotGirlNextDoor,fairytale808,pink and blonde (iktr),blonde woman in pink bikinisuit standing on a ...,0849c7aafe126ad27b58cb6e6c9c6592,/r/HotGirlNextDoor/comments/1003dod/pink_and_b...,https://i.redd.it/x56eekrd8c9a1.jpg,1003dod.jpg,data/image/1003dod.jpg,SexyDiffusion,True,True,True,[]
10044wv,a train on a bridge over a body of water,0.38273,10044wv,4032.0,3024.0,CityPorn,dawginphilly,"Wissahickon, Phila, PA",arafed view of a train on a bridge over a river,abb87d7618eec7cd6a86d7647263ea1b,/r/CityPorn/comments/10044wv/wissahickon_phila...,https://i.redd.it/4y6hzocyec9a1.jpg,10044wv.jpg,data/image/10044wv.jpg,CityDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13m91m2,,,13m91m2,,,DLAH,qwertybird3141,College grad,arafed woman standing on steps with a red and ...,adcbd61b5e37c5e048cb6907eca21bfe,/r/DLAH/comments/13m91m2/college_grad/,https://i.redd.it/skv5m0jwiw0b1.jpg,13m91m2.jpg,data/image/13m91m2.jpg,PrettyGirlDiffusion,True,True,True,[]
13gwhao,,,13gwhao,,,bathandbodyworks,becki5926,💙Coupons Are Here❤️,a close up of a sign advertising bath and body...,d4ba98857b64b35ecec420a6b7e32f97,/r/bathandbodyworks/comments/13gwhao/coupons_a...,https://i.redd.it/pl5orr6d8qza1.jpg,13gwhao.jpg,data/image/13gwhao.jpg,CandleDiffusion,True,True,False,[]
13m6gm7,,,13m6gm7,,,bathandbodyworks,unicornkenz,Candle Sale,bath and body works coup coup,47a78b117b59b636fa22093ec4f29bd7,/r/bathandbodyworks/comments/13m6gm7/candle_sale/,https://i.redd.it/h321gmgf0w0b1.jpg,13m6gm7.jpg,data/image/13m6gm7.jpg,CandleDiffusion,True,True,True,[]


CPU times: total: 469 ms
Wall time: 1.44 s


In [11]:
%%time

merged_captions = pandas.concat([new_captions, captions])
merged_captions.set_index(keys=['id', 'dense_captions_text', 'dense_captions_confidence'], inplace=True, drop=False)
merged_captions.drop_duplicates(inplace=True)
merged_captions.reset_index(drop=True, inplace=True)

display(f'{merged_captions.shape[0] - captions.shape[0]} new rows added to captions')

merged_captions.to_parquet('data/parquet/image_captions.parquet', filesystem=file_system, engine='pyarrow')
# display(pandas.read_parquet('data/parquet/image_captions.parquet', filesystem=file_system, engine='pyarrow'))

'0 new rows added to captions'

,dense_captions_text,dense_captions_confidence,dense_captions_boundingBox.x,dense_captions_boundingBox.y,dense_captions_boundingBox.w,dense_captions_boundingBox.h,id
0,a pier in the ocean,0.321105,0,0,4032,3024,1002bsl
1,a beach with a pier and buildings,0.339972,0,0,3994,2940,1002bsl
2,a cloudy sky over a field,0.312660,0,0,3978,1475,1002bsl
3,a long shot of a pier,0.524807,1129,1313,2855,412,1002bsl
4,a beach with waves and a boat,0.345760,0,1669,3986,1282,1002bsl
...,...,...,...,...,...,...,...
59097,a building with a roof,0.248191,772,658,190,174,zzy49v
59098,a bridge over a cliff,0.359007,0,553,1042,754,zzy49v
59099,a white building with windows,0.392007,0,460,112,230,zzy49v
59100,a white building with many windows,0.446609,925,519,154,791,zzy49v


CPU times: total: 1.06 s
Wall time: 3.77 s


In [12]:
%%time

merged_tags = pandas.concat([new_tags, tags])
merged_tags.set_index(keys=['id', 'tags_name', 'tags_confidence'], inplace=True, drop=False)
merged_tags.drop_duplicates(inplace=True)
merged_tags.reset_index(drop=True, inplace=True)

display(f'{merged_tags.shape[0] - tags.shape[0]} new rows added to tags')

merged_tags.to_parquet('data/parquet/image_tags.parquet', filesystem=file_system, engine='pyarrow')
# display(pandas.read_parquet('data/parquet/image_tags.parquet', filesystem=file_system, engine='pyarrow'))

'0 new rows added to tags'

,tags_name,tags_confidence,id
0,outdoor,0.997431,1002bsl
1,cloud,0.995743,1002bsl
2,sky,0.992414,1002bsl
3,water,0.982002,1002bsl
4,beach,0.943822,1002bsl
...,...,...,...
121914,building,0.947034,zzy49v
121915,waterfall,0.917590,zzy49v
121916,mountain,0.871428,zzy49v
121917,nature,0.845276,zzy49v


CPU times: total: 922 ms
Wall time: 2.51 s


In [13]:
%%time

merged_crops = pandas.concat([new_crops, crops])
merged_crops.set_index(keys=['id'], inplace=True, drop=False)
merged_crops.drop_duplicates(inplace=True)
merged_crops.reset_index(drop=True, inplace=True)

display(f'{merged_crops.shape[0] - crops.shape[0]} new rows added to crops')

merged_crops.to_parquet('data/parquet/image_cropping.parquet', filesystem=file_system, engine='pyarrow')

display(pandas.read_parquet('data/parquet/image_cropping.parquet', filesystem=file_system, engine='pyarrow'))

'0 new rows added to crops'

,smart_crop_aspectRatio,smart_crop_boundingBox.x,smart_crop_boundingBox.y,smart_crop_boundingBox.w,smart_crop_boundingBox.h,id
0,1.0,619,71,2737,2737,1002bsl
1,1.0,206,0,797,797,1002cgj
2,1.0,0,140,1462,1462,1002cx2
3,1.0,143,138,1249,1250,1003dod
4,1.0,206,0,3013,3012,10044wv
...,...,...,...,...,...,...
6609,1.0,0,0,2273,2273,zzu8i8
6610,1.0,30,63,1248,1248,zzv4hz
6611,1.0,135,0,1299,1299,zzvv3x
6612,1.0,101,101,970,970,zzxq2l


CPU times: total: 172 ms
Wall time: 533 ms


In [14]:
import os

def create_thumbnail(_image_id: str, _curated_df: pandas.DataFrame, _crops: pandas.DataFrame, _extant_file_names: list):
	_file_system: AzureBlobFileSystem = AzureFileStorageAdapter('data').get_file_storage()
	try:
		if _image_id is None or _image_id in _extant_file_names:
			display(f'Image {_image_id} already exists, skipping', clear=True)
			return None

		cropping_information = _crops.loc[_crops['id'] == _image_id]
		if cropping_information is None or len(cropping_information) == 0:
			display(f'No cropping information for {_image_id}, skipping', clear=True)
			return None

		record = _curated_df.loc[_curated_df['id'] == _image_id]
		record_path = record.to_dict(orient='records')[0]['path']
		image_url = file_system.url(record_path)
		original_image = Image.open(requests.get(image_url, stream=True).raw)
		copied_image = original_image.copy()
		original_image.close()

		cropped = copied_image.crop((cropping_information['smart_crop_boundingBox.x'].values[0],
									 cropping_information['smart_crop_boundingBox.y'].values[0],
									 cropping_information['smart_crop_boundingBox.x'].values[0] +
									 cropping_information['smart_crop_boundingBox.w'].values[0],
									 cropping_information['smart_crop_boundingBox.y'].values[0] +
									 cropping_information['smart_crop_boundingBox.h'].values[0]))
		copied_image.close()

		resized = cropped.resize((512, 512), 1)
		resized.save('temp.jpg')
		resized.close()
		file_system.upload('temp.jpg', f'data/image/thumbnail/{_image_id}.jpg', overwrite=True)
		display(f'Thumbnail created for {_image_id}', clear=True)
		return None

	except Exception as ex:
		display(f'Error creating thumbnail for {_image_id}: {ex}', clear=True)
		return None

In [15]:
%%time

extant = [os.path.basename(item.replace('\n', '').split('.')[0]) for item in file_system.ls('data/image/thumbnail')]
display(len(extant), clear=True)

['1002bsl',
 '1002cgj',
 '1002cx2',
 '1003dod',
 '10044wv',
 '1004bvl',
 '1005neb',
 '1007wlw',
 '1007zxc',
 '1008d22',
 '1008ddt',
 '1008fqh',
 '1008jy5',
 '1008zf0',
 '1009o0q',
 '100b2b1',
 '100c3hk',
 '100c6hd',
 '100cjga',
 '100dbjr',
 '100dcrd',
 '100ed6o',
 '100efeg',
 '100fzdq',
 '100gpoh',
 '100h3yk',
 '100hk8p',
 '100hpjy',
 '100ilud',
 '100j0e2',
 '100jl1b',
 '100km4t',
 '100knfl',
 '100lihz',
 '100lj2b',
 '100ls5q',
 '100ltto',
 '100m3g5',
 '100n1xj',
 '100nibu',
 '100nndx',
 '100p2f8',
 '100pjjw',
 '100q35t',
 '100q7x9',
 '100qaru',
 '100qk25',
 '100qpap',
 '100qx25',
 '100rfwd',
 '100rhce',
 '100riou',
 '100rsun',
 '100rynl',
 '100rzx6',
 '100s0s5',
 '100s2p8',
 '100sez9',
 '100shj4',
 '100sjzi',
 '100sk7s',
 '100sk85',
 '100tlqk',
 '100uex5',
 '100uo7z',
 '100urox',
 '100v058',
 '100vc0h',
 '100vvqp',
 '100w0ff',
 '100w2ga',
 '100whmx',
 '100x04y',
 '100xd94',
 '100xi10',
 '100xqd2',
 '100yocw',
 '100z682',
 '100zb62',
 '100zmxh',
 '100zrmf',
 '1010qb3',
 '1011gjo',
 '10

CPU times: total: 10.5 s
Wall time: 19.1 s


In [16]:
%%time

foo = curated_df.loc[curated_df['accept'] == True]
foo = foo.loc[foo['id'].isin(crops.id.unique())]
foo = foo.loc[~foo['id'].isin(extant)]
foo.dropna(inplace=True)
foo.reset_index(inplace=True, drop=True)

# display(foo)
# display(crops)
id_records = foo.id.values.tolist()
# display(len(id_records))
for item in tqdm(id_records, total=len(id_records), desc='Creating thumbnails'):
	create_thumbnail(item, curated_df, crops, extant)

all_images = file_system.ls('data/image/thumbnail')
old_images = extant['id'].values.tolist()
new_images = [item for item in all_images if item not in old_images]
display(new_images)
display(len(new_images))

Creating thumbnails: 0it [00:00, ?it/s]

CPU times: total: 31.2 ms
Wall time: 64 ms


In [17]:
!jupyter notebook stop

^C


Error processing line 1 of D:\code\repos\simple-collection\venv\lib\site-packages\vision-1.0.0-py3.9-nspkg.pth:

  Traceback (most recent call last):
    File "C:\Users\AJ Stangl\AppData\Local\Programs\Python\Python39\lib\site.py", line 169, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 562, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored
Error processing line 1 of D:\code\repos\simple-collection\venv\lib\site-packages\vision-1.0.0-py3.9-nspkg.pth:

  Traceback (most recent call last):
    File "C:\Users\AJ Stangl\AppData\Local\Programs\Python\Python39\lib\site.py", line 169, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 562, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored
Error processing line 1 of D:\code\repos\simple-co